In [1]:
import os
import copy
import string
import re
import pickle
from collections import OrderedDict
import time

### Identify the annotated span of text (Old version)
    a. highlighter contains the combs of the {key=concept : value=span{s : e}}

In [2]:
# Input: some candidate concepts matched to the sentence;
# requirment: 1. must find the concept (or combination of concepts) that contains maximum words
#             2. if two options has same maximum words: we choose the one has fewest concepts

# CXR showed large R pleural effusion with ?

# 12	16	large	
# 18	35	R pleural effusion	
# 18	19	R
# 20	35	pleural effusion	
# 20	26	pleural	      
# 28	35	effusion	

# option 1: large | R pleural effusion  -> best
# option 2: large | R | pleural effusion
# option 3: large | R | pleural | effusion

# text case 4:

# need to follow up + rheumatology clinic + as an outpatient  9words 3concepts good


# need + follow up in reheumatology clinic + as an outpatient 9words 3concepts good

# need + follow up + rheumatology clinic + as an outpatient 8


highlighter = OrderedDict()
def pair_matched_concept_indexes(case, line, path):
    """
      Args: case (the EHR text)
            line (the sentence of the case)
            path (the row data contains the all the matched concepts of each sentence of the case)
                  [0]: the case_id;
                  [2][3]: the start and end index of the matched concept in the text
                  [4]: the matched concept
                  [5]: the sentence
      Return: Ordered dictionary; ex: {[3, 8]: apple}; {[3, 13]: apple tree} {[10, 13]: tree}
    """
    highlighter.clear()
    with open(path, 'r', encoding='utf-8') as annotation_file:
        for annotate in annotation_file:
            text_to_annotate = annotate.split("\t")[5]
            text_to_annotate = re.sub(r'\s+', '', text_to_annotate)
        if case == annotate.split("\t")[0] and line == text_to_annotate:
            span = (int(annotate.split("\t")[2]), int(annotate.split("\t")[3]))
            highlighter[span] = annotate.split("\t")[4]


In [3]:
# Using Fansy Annotation Styly
annotation_style = "<style> \
    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300;400;500;600;700&display=swap'); \
    @import url('https://fonts.googleapis.com/css2?family=Vistol Regular:wght@300;400;500;600;700&display=swap'); \
    \
    .spark-nlp-display-scroll-entities {\
        border: 1px solid #E7EDF0;\
        border-radius: 3px;\
        text-align: justify;\
        \
    }\
    .spark-nlp-display-scroll-entities span {  \
        font-size: 14px;\
        line-height: 24px;\
        color: #536B76;\
        font-family: 'Montserrat', sans-serif !important;\
    }\
    \
    .spark-nlp-display-entity-wrapper{\
    \
        display: inline-grid;\
        text-align: center;\
        border-radius: 4px;\
        margin: 0 2px 5px 2px;\
        padding: 1px\
    }\
    .spark-nlp-display-entity-name{\
        font-size: 14px;\
        line-height: 24px;\
        font-family: 'Montserrat', sans-serif !important;\
        \
        background: #f1f2f3;\
        border-width: medium;\
        text-align: center;\
        \
        font-weight: 400;\
        \
        border-radius: 5px;\
        padding: 2px 5px;\
        display: block;\
        margin: 3px 2px;\
    \
    }\
    .spark-nlp-display-entity-type{\
        font-size: 14px;\
        line-height: 24px;\
        color: #ffffff;\
        font-family: 'Montserrat', sans-serif !important;\
        \
        text-transform: uppercase;\
        \
        font-weight: 500;\
\
        display: block;\
        padding: 3px 5px;\
    }\
    \
    .spark-nlp-display-entity-resolution{\
        font-size: 14px;\
        line-height: 24px;\
        color: #ffffff;\
        font-family: 'Vistol Regular', sans-serif !important;\
        \
        text-transform: uppercase;\
        \
        font-weight: 500;\
\
        display: block;\
        padding: 3px 5px;\
    }\
    \
    .spark-nlp-display-others{\
        font-size: 14px;\
        line-height: 24px;\
        font-family: 'Montserrat', sans-serif !important;\
        \
        font-weight: 400;\
    }\
\
</style>"

### Best Combination of Concepts Algorithm

In [4]:
def check_maximum(cur_words, max_words, temp, result):
    print('temp: ', temp)
    if (cur_words == max_words[0] and len(temp) <= len(result)) or cur_words > max_words[0]:
        max_words[0] = cur_words
        result.clear()
        for k, v in temp.items():
            # print(k, ':', v)
            result[k] = v
''' idx: the index of a concept in the concept_map
    concept_map: a dict: {{1 : 5} = 'words' ; {1 : 11} = 'words length'}
    cur_words: integer the # of words 
    max_words: an array: to record and update the max number of annotated words
    temp: dict: {{1 : 11} = 'words length'}
    result: a dict:
    word_cnt: an array: record the number of words in each value of concept_map
'''
'''
idx = 0

concept_map:
0: [12 : 15] = mid
1: [12 : 21] = mid fluid
2: [12 : 30] = mid fluid overload
3: [17 : 30] = fluid overload

cur_words = 0

max_words = [0]

word_cnt = [1, 2, 3, 2]
'''


def dfs_maximum_annotates(sorted_concepts, current_concept_index, num_concepts, last_taken_concept, total_concepts_used, total_words_collected, result, cur_ans, final_ans):
    if current_concept_index == num_concepts:
        max_words_collected = result[0]
        min_concepts_used = result[1]
        if total_words_collected > max_words_collected:
            max_words_collected = total_words_collected
            min_concepts_used = total_concepts_used
            final_ans.clear()
            final_ans.extend(cur_ans)
        elif total_words_collected == max_words_collected and total_concepts_used < min_concepts_used:
            min_concepts_used = total_concepts_used
            final_ans.clear()
            final_ans.extend(cur_ans)
            
        result[0] = max_words_collected
        result[1] = min_concepts_used
        return
    
    
    can_take = (last_taken_concept == -1) or (sorted_concepts[last_taken_concept][1] < sorted_concepts[current_concept_index][0])
    #taken
    new_ans = cur_ans.copy()
    if can_take: 
        new_ans.append(current_concept_index)
        dfs_maximum_annotates(sorted_concepts, current_concept_index + 1, num_concepts,
                        current_concept_index, total_concepts_used + 1, 
                        total_words_collected + sorted_concepts[current_concept_index][2], 
                        result, new_ans, final_ans)
    #not taken
    new_ans = cur_ans.copy()
    dfs_maximum_annotates(sorted_concepts, current_concept_index + 1, num_concepts,
                        last_taken_concept, total_concepts_used, total_words_collected, 
                        result, new_ans, final_ans)
    

In [5]:
def dfs_dp_maximum_annotates(sorted_concepts, current_concept_index, num_concepts, last_taken_concept, dp):
    if current_concept_index == num_concepts:
        return 0, 0, []
        
    dp_if_not_taken = (0, 0)
    dp_if_taken = (0, 0)
    
    
    if dp[current_concept_index][1] == None:
        future_data = dfs_dp_maximum_annotates(sorted_concepts, current_concept_index + 1, num_concepts,
                            current_concept_index,  
                            dp)
        dp[current_concept_index][1] = future_data[0], future_data[1], future_data[2]
    
    dp_if_not_taken = dp[current_concept_index][1]    
        
    if dp[current_concept_index][0] == None:
        next_possible_concept_index = current_concept_index + 1
        while next_possible_concept_index < num_concepts and sorted_concepts[next_possible_concept_index][0] <= sorted_concepts[current_concept_index][1]:
            next_possible_concept_index += 1

        future_data = dfs_dp_maximum_annotates(sorted_concepts, next_possible_concept_index, num_concepts,
                        current_concept_index,  
                        dp)
        future_concept_list = [current_concept_index]
        if future_data[2] is not None:
            future_concept_list.extend(future_data[2])
            
        dp[current_concept_index][0] = sorted_concepts[current_concept_index][2] + future_data[0], 1 + future_data[1], future_concept_list

    dp_if_taken = dp[current_concept_index][0]
    
    #print(dp)
     
    future_words_collected_if_taken = dp_if_taken[0]
    future_words_collected_if_not_taken = dp_if_not_taken[0]
    future_concepts_used_if_taken = dp_if_taken[1]
    future_concepts_used_if_not_taken = dp_if_not_taken[1]
    future_concept_list_if_taken = dp_if_taken[2]
    future_concept_list_if_not_taken = dp_if_not_taken[2]
    
    #print(current_concept_index, future_words_collected_if_taken, future_words_collected_if_not_taken, future_concepts_used_if_taken, future_concepts_used_if_not_taken)
    
    if (future_words_collected_if_taken > future_words_collected_if_not_taken) or (future_words_collected_if_taken == future_words_collected_if_not_taken and future_concepts_used_if_taken < future_concepts_used_if_not_taken):
        return future_words_collected_if_taken, future_concepts_used_if_taken, future_concept_list_if_taken
    else:
        return future_words_collected_if_not_taken, future_concepts_used_if_not_taken, future_concept_list_if_not_taken


In [6]:
def find_best_combination_of_annotates_2(sorted_concept_list):
    start = time.time()
    
    num_concepts = len(sorted_concept_list)
    dp = []
    for i in range(num_concepts):
        dp.append([None, None])
    result = dfs_dp_maximum_annotates(sorted_concept_list, 0, num_concepts, -1,  dp)
   
    max_words_collected = result[0]
    min_concepts_used = result[1]
    best_concept_list = result[2]
    #print(max_words_collected, min_concepts_used, best_concept_list)
    
    end = time.time()
    #print("time 2", end - start)
    
    return max_words_collected, min_concepts_used, best_concept_list, (end - start)

In [7]:
def find_best_combination_of_annotates(sorted_concept_list):
    start = time.time()
    
    num_concepts = len(sorted_concept_list)
    result = [0, 9999]
    cur_ans = []
    final_ans = []
    dfs_maximum_annotates(sorted_concept_list, 0, num_concepts, -1, 0, 0, result, cur_ans, final_ans)
    for i in final_ans:
        print(sorted_concept_list[i][3])
    max_words_collected = result[0]
    min_concepts_used = result[1]
    print(max_words_collected, min_concepts_used)
    
    end = time.time()
    print("time 1", end - start)
    return final_ans

In [8]:
# "The patient was taken on to the operating room for a redo mitral 
# valve surgery with mitral valve replacement"
def annotated_single_case(input_mgrep, output_html):
    concept_list = []
    cur_sent = ""
    final_concept_map = None
    with open(input_mgrep, 'r') as f:
        with open(annotation_html, 'w') as out_file:
            out_file.write('<html>')
            out_file.write('<body>')
            out_file.write(annotation_style)
            for line in f:
                temp_line = line.split('\t')
                cur_sent = temp_line[5]
                #print(temp_line[4])
                concept_list.append((int(temp_line[2]), int(temp_line[3]), len(temp_line[4].split()), temp_line[4]))
                        
            sorted_concept_list = sorted(concept_list)
            #print(cur_sent)
            #print(sorted_concept_list)
#             for aa in sorted_concept_list:
#                 print(aa[3])
            
            #find_best_combination_of_annotates(sorted_concept_list)
            #find_best_combination_of_annotates_2(sorted_concept_list)
                
            ## Prepare the result
            final_concept_map = find_best_combination_of_annotates_2(sorted_concept_list)

            
            # Write the result to a HTML file
            i = 0
            start = 0
            end = 0
            sentence = temp_line[5]
            entity_type = 'CIT'       
            while i < len(sentence):
                for s in final_concept_map[2]:
                    if(i >= len(sentence) - 1):
                        break
                    concept = sorted_concept_list[s]
                    start_index = concept[0] - 1
                    end_index = concept[1] - 1
                    if start_index <= i <= end_index:
#                         print(start_index)
                        out_file.write('<span class="spark-nlp-display-entity-wrapper" style="background-color: #800080"> \
                                       <span class="spark-nlp-display-entity-name">%s</span> \
                                       <span class="spark-nlp-display-entity-type">%s</span></span>' %(sentence[start_index:end_index+1], entity_type))
                        i = i + len(sentence[start_index:end_index+1])
                # Bug is here, should go through the rest text outside of for loop, to avoid the overwrite the plain text
                out_file.write('<span class="spark-nlp-display-others" style="background-color: white">%s</span>' %sentence[i])
                i += 1
            out_file.write('<div>')
            out_file.write('<br>')
            out_file.write('</body>')
            out_file.write('</html>')
        out_file.close()
    
    number_of_words = len((re.sub(' +', ' ', cur_sent)).strip().split(' '))
    return final_concept_map[0], final_concept_map[1], number_of_words, final_concept_map[3]
    # Saving the result
    # with open('processed_result.txt', 'w') as write:
    #     for file_k, file_v in file_map.items():
    #         for sent_id in file_v:
    #             for concept_k, concept_v in concept_map_new[sent_id].items():
    #                 write.write(str(file_k) + '\t' + str(concept_k[0]) + '\t' + str(concept_k[1]) + '\t' + str(concept_v) + '\t' + str(sent_map[sent_id]))


In [9]:
index = 5
mgrep_matched_result = "CDIT_C1_dfs_one_case_"+str(index)+".txt"
annotation_html = 'sent_' + str(index) + '_new.html'
annotated_single_case(mgrep_matched_result, annotation_html)

(10, 3, 16, 0.0)

In [10]:
def main():
    input_case_filename = 'CDIT_C3_clean_091324.txt'
    
    total_word_count = 0
    anno_word_count = 0
    num_annotations = 0
    file_index = 0

          # open one of the case
    with open(input_case_filename, 'r', encoding='utf-8') as readfile:
        lines_cur_file = []
        last_sentence = ''
        for line in readfile:
            temp_line = line.split('\t')
            #print(line)
            if last_sentence == '' or temp_line[5] == last_sentence:
                lines_cur_file.append(line)
            else:    
                with open('test_folder\\' + str(file_index) + ".txt", "w") as file:
                    #print("all", lines_cur_file)
                    if len(lines_cur_file) > 0:
                        file.writelines(lines_cur_file)
                        file_index += 1
                    lines_cur_file = [line]
                    #print(file_index)
            last_sentence = temp_line[5]
    print(file_index)
    

In [11]:
main()

2791


In [17]:
folder_name = 'test_folder'
out_folder_name = 'out_test_folder'
file_names = os.listdir(folder_name)
total_time = 0
total_words_count = 0
num_anno_count = 0
num_concepts = 0
num_files = 0
for file in file_names:
    print(file)
    #if file != '1895.txt':
    #    continue
    annotation_html = out_folder_name + "\\" 'sent_' + file + '_new.html'
    file_path = folder_name + "\\" + file
    res = annotated_single_case(file_path, annotation_html)
    num_anno_count += res[0]
    num_concepts += res[1]
    total_words_count += res[2]
    total_time += res[3]
    num_files += 1
    print(num_anno_count, num_concepts, total_words_count)
    
print(num_anno_count / total_words_count * 100)
print(num_anno_count / num_concepts)
print(total_time)

0.txt
2 1 8
1.txt
12 6 28
10.txt
15 8 44
100.txt
17 9 58
1000.txt
18 10 63
1001.txt
22 11 73
1002.txt
26 12 79
1003.txt
28 13 86
1004.txt
32 14 91
1005.txt
34 15 97
1006.txt
36 16 102
1007.txt
37 17 105
1008.txt
38 18 108
1009.txt
40 19 110
101.txt
46 21 124
1010.txt
48 22 126
1011.txt
50 24 132
1012.txt
53 25 135
1013.txt
54 26 139
1014.txt
59 27 144
1015.txt
61 28 147
1016.txt
64 29 150
1017.txt
66 30 152
1018.txt
73 35 169
1019.txt
92 42 195
102.txt
95 44 212
1020.txt
104 49 231
1021.txt
107 52 244
1022.txt
112 55 254
1023.txt
125 60 272
1024.txt
130 64 285
1025.txt
133 66 298
1026.txt
140 68 306
1027.txt
145 70 314
1028.txt
151 72 330
1029.txt
154 74 341
103.txt
157 76 371
1030.txt
163 79 384
1031.txt
170 82 400
1032.txt
172 84 409
1033.txt
174 85 424
1034.txt
187 89 442
1035.txt
194 91 450
1036.txt
199 95 467
1037.txt
206 99 476
1038.txt
210 101 481
1039.txt
222 103 496
104.txt
227 104 507
1040.txt
230 106 523
1041.txt
231 107 529
1042.txt
238 111 547
1043.txt
247 114 568
1044.txt

In [13]:
# file_map = OrderedDict()
# sent_map = OrderedDict()
# concept_map = OrderedDict()

# # "The patient was taken on to the operating room for a redo mitral 
# # valve surgery with mitral valve replacement"

# mgrep_matched_result = "CDIT_C1_dfs_one_case_1.txt"
# annotation_html = 'sent_1.html'

# def annotated_single_case(input_mgrep, output_html):

#     with open(mgrep_matched_result, 'r') as f:
#         with open(annotation_html, 'w') as out_file:
#             out_file.write('<html>')
#             out_file.write('<body>')
#             out_file.write(annotation_style)
#             cur_sent = ""
#             cur_sent_id = -1
#             for line in f:
#                 temp_line = line.split('\t')
#                 # decide the string id
#                 if temp_line[5] != cur_sent:
#                     cur_sent = temp_line[5]
#                     cur_sent_id += 1
#                     # fill the file_map
#                     if temp_line[0] not in file_map.keys():
#                         file_map[temp_line[0]] = list()
#                     file_map[temp_line[0]].append(cur_sent_id)
#                     # fill the sent_map
#                     sent_map[cur_sent_id] = cur_sent
#                     # create new concept_map item
#                     concept_map[cur_sent_id] = OrderedDict()
#                 # file the conccept_map
#                 temp_concept_map = concept_map[cur_sent_id]
#                 temp_concept_map[(temp_line[2], temp_line[3])] = temp_line[4]
            
            
#             ## Prepare the result
#             concept_map_new = OrderedDict() 
#             for k, v in concept_map.items():
#                 temp_v = OrderedDict(sorted(v.items(), key=lambda x: x[0]))
#                 # print('new', temp_v)
#                 concept_map_new = find_best_combination_of_annotates(temp_v)
#                 sorted_spans = sorted(concept_map_new.keys())
#                 sorted_spans_matched = [(int(i[0])-1, int(i[1])-1) for i in sorted_spans]

#             # sorted_spans_matched is the "list of best combination of pairs of (start : end)"
#             print(concept_map_new) 
            
#             # Write the result to a HTML file
#             i = 0
#             start = 0
#             end = 0
#             sentence = temp_line[5]
#             entity_type = 'CIT'       
#             while i < len(sentence):
#                 for s in sorted_spans_matched:
#                     if(i >= len(sentence) - 1):
#                         break

#                     if s[0] <= i <= s[1]:
#                         out_file.write('<span class="spark-nlp-display-entity-wrapper" style="background-color: #800080"> \
#                                        <span class="spark-nlp-display-entity-name">%s</span> \
#                                        <span class="spark-nlp-display-entity-type">%s</span></span>' %(sentence[s[0]:s[1]+1], entity_type))
#                         i = i + len(sentence[s[0]:s[1]+1])
#                 # Bug is here, should go through the rest text outside of for loop, to avoid the overwrite the plain text
#                 out_file.write('<span class="spark-nlp-display-others" style="background-color: white">%s</span>' %sentence[i])
#                 i += 1
#             out_file.write('<div>')
#             out_file.write('<br>')
#             out_file.write('</body>')
#             out_file.write('</html>')
#         out_file.close()
        
#     # Saving the result
#     # with open('processed_result.txt', 'w') as write:
#     #     for file_k, file_v in file_map.items():
#     #         for sent_id in file_v:
#     #             for concept_k, concept_v in concept_map_new[sent_id].items():
#     #                 write.write(str(file_k) + '\t' + str(concept_k[0]) + '\t' + str(concept_k[1]) + '\t' + str(concept_v) + '\t' + str(sent_map[sent_id]))


In [14]:
annotated_single_case(mgrep_matched_result, annotation_html)

(10, 3, 16, 0.0)

### Batch Runing

In [15]:
def main():
    input_case_path = '/Users/shuxinzhou/Desktop/ SABOC/Cardiology/annotation_algo/data/splitted_train_data_150/'
    html_folder_path = '/Users/shuxinzhou/Desktop/ SABOC/Cardiology/annotation_algo/data/cardiology_annotation_html_0805/'
    mgrep_matched_result = '/Users/shuxinzhou/Desktop/ SABOC/Cardiology/annotation_algo/data/CDIT_A3_mgrep_match_clean_080524.txt'

    long_sent_count = 0

    stat_file = open('/Users/shuxinzhou/Desktop/ SABOC/Cardiology/annotation_algo/data/coverage_states_result.txt', 'w')
    stat_file.write('File name\tTotal word count\tAnnotated word count\tCoverage\tBreadth\tNumber of annotations\n')

    # 500 cases input
    for filename in os.listdir(input_case_path):
        total_word_count = 0
        anno_word_count = 0
        num_annotations = 0

    # file_map = dict() # key: file_name, value: sent_id list
    # sent_map = dict() # key: sent_id, value: sent_string
    # concept_map = dict() # key: sent_id, value: OrderedDict(k: (0, 5), v: "hello")
    # concept_map_new = OrderedDict()  # concept_map, removed unneeded concept, key: sent_id, value:

    # check if the case is empty or not
        if os.stat(input_case_path + filename).st_size != 0:
          # open one of the case
          with open(input_case_path + filename, 'r', encoding='utf-8') as readfile:
                print('****** opening', filename)
                wrt_fname = filename.split('.')[0] + '.html'
                # create html file in the folder and write a html file
                with open(html_folder_path + wrt_fname, 'w', encoding='utf-8') as out_file:
                    out_file.write('<html>')
                    out_file.write('<body>')
                    out_file.write(annotation_style)

                    for line in readfile:
                        if len(line) > 1:
                            text_annotate = line
                            try:
                                text_annotate = re.sub(r'\s+', '', text_annotate)
                                # print("text source: " + text_annotate)
                                pair_matched_concept_indexes(filename, text_annotate, mgrep_matched_resul)


                              # sort valid_high based on start index and end index
                                valid_high = OrderedDict(sorted(highlighter.items(), key=lambda x: x[0]))
                                valid_high_res = find_best_combination_of_annotates(valid_high)

                                sorted_spans = sorted(valid_high_res.keys())
                                sorted_spans_matched = [(int(i[0]) - 1, int(i[1]) - 1) for i in sorted_spans]
                                print(sorted_spans_matched)

                                total_word_count += len((re.sub(' +', ' ', line)).strip().split(' '))
                              # total_word_count += len(line.split(' '))
                                for k in valid_high_res:
                                    split_anno_word = re.sub('/', ' ', valid_high_res[k])
                                    anno_word_count += len(split_anno_word.split(' '))
                                num_annotations += len(valid_high_res)

                                word = ''
                                i = 0
                                count = 0
                                while i < len(line):
                                    flag = False
                                    for s in sorted_spans_matched:
                                        word = line[s[0]:s[1] + 1]
                                        if s[0] <= i <= s[1]:
                                            flag = True
                                            break
                                    if flag is True:
                                        count += 1
                                        if count % 2 == 0:
                                            out_file.write('<span class="spark-nlp-display-entity-wrapper" style="background-color: #768256"><span class="spark-nlp-display-entity-name"><b>%s</b></span>' % word)
                                        else:
                                            out_file.write('<span class="spark-nlp-display-entity-wrapper" style="background-color: #768256"><span class="spark-nlp-display-entity-name"><b>%s</b></span>' % word)
                                        i = i + len(word)
                                    else:
                                        out_file.write('%s' % line[i])
                                        i += 1
                            except:
                                long_sent_count += 1
                        # print('error')
                                pass
                        # out_file.flush()
                        else:
                            out_file.write('<br>')
                    out_file.write('</body>')
                    out_file.write('</html>')
                # out_file.close()
                readfile.close()
                print('Total word count =', total_word_count)
                print('Annotated word count =', anno_word_count)
                Coverage = (anno_word_count / total_word_count) * 100
                print('Coverage =', Coverage)
                if num_annotations > 0:
                    breadth = anno_word_count / num_annotations
                    print('Breadth =', breadth)
                else:
                    breadth = 0
                print()
                stat_file.write(
                  filename + '\t' + str(total_word_count) + '\t' + str(anno_word_count) + '\t' + str(Coverage) + '\t' + str(
                      breadth) + '\t' + str(num_annotations) + '\n')


In [16]:
if __name__=="__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/shuxinzhou/Desktop/ SABOC/Cardiology/annotation_algo/data/coverage_states_result.txt'